# Prepare a CSV file for the OWG

This notebook contains code as a function and a walk through for creating a csv file that can be use in optical wave gauging

In [5]:
# import packages and libs

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import datetime
import calendar
from scipy import interpolate
import os
import shutil
import csv
import pytz


## Read data

Data comes from two sources:  
* 1. NDBC standard meteorlogical observation archives, found [here](https://www.ndbc.noaa.gov/) 
* 2. Archived ReCON imagery, partial records can be scraped from the [website](https://www.glerl.noaa.gov/metdata/) or accessed from GLERL's network


This data must be joined so that each webcam image is associated with the meterological observations taken no later than 30 minutes before or after the image was captured.

In [28]:
# Complete csv

def waveframetocsv(fn, target, csvfile, directory, localtimezone):
    '''
    This function takes a txt file and folder of OWG images and combines the data into a csv file that can be fed into the OWG. 
    version 2.0
    
    fn: filename and location of .txt file
    target: attribute that the owg will be predicting
    csvfile:the name of the csvfile being created
    directory: the directory of images that have been prepped for OWG filtering
    localtimezone: the timezone code in pytz that the images were taken in
    '''
    
    #counters for debugging and perfomance info
    initaladdcounter = 0
    rangecounter = 0
    successcounter = 0
    failcounter = 0
    timecounter = 0
    outofrangecounter = 0
    
    # create a dataframe from the buoy data
    df = pd.read_csv(fn, skiprows=range(1,2), delim_whitespace = True, \
                    parse_dates={'date':[0,1,2,3,4]}, keep_date_col=False)

    # Transfer data in "date" column to a column where it is stored as a datetime object
    df['datetime'] = pd.to_datetime(df['date'], format = '%Y %m %d %H %M',utc=True)
    df = df.drop(df.columns[[0,1,2,3,6,8,9,10,11,12, 13]], axis = 1)
    
    # calculate unix datetime
    df['epoch']=(df['datetime'] - pd.Timestamp("1970-01-01",tz='utc')) // pd.Timedelta('1s')
    
    print(df)
    
    # delete the csv file if it exsists
    try:
        print ("Overwriting csv file")
        os.remove(csvfile)
        owgframe = pd.DataFrame({"id":[], "H":[], "T":[], "MWDIR":[]})
    except:
        print("couldn't find file, making new one")
        # create csv file that will be appended to by loop
        owgframe = pd.DataFrame({"id":[], "H":[], "T":[], "MWDIR":[]})

    #loop through directory and extract unix timestamp
    for filename in os.listdir(directory):
        # Use string slicing to remove .jpg from filename
        size = len(filename)
        fn = filename[:size - 4]
       
        
        #get utc time from filename
        local = pytz.timezone(localtimezone)
        naive = datetime.datetime.strptime(fn, "%Y%m%d%H%M")
        local_dt = local.localize(naive)
        utc_dt = local_dt.astimezone(pytz.utc)
        utime = calendar.timegm(utc_dt.timetuple())
        
        # combine datasets
        try:
            # track if buoy was in the water when the image was taken and find the closest record
            if utime >= df['epoch'].iat[0]:
                result_index = df['epoch'].sub(utime).abs().idxmin()
                result = df['epoch'].iat[result_index]
                rangecounter = rangecounter + 1
                
                # determine if the closest record was within 30 minutes and add to final dataframe
                if abs(utime - result) <= 1800:
                    initaladdcounter = initaladdcounter + 1
                    stageframe = {"id":filename, "H": df['WVHT'].iat[result_index], "T": df['DPD'].iat[result_index], "MWDIR":df['MWD'].iat[result_index]}
                    owgframe = owgframe.append(stageframe, ignore_index = True)
                    
                    # drop NAN values for the target variable
                    if target == "WVHT":
                        owgframe = owgframe[owgframe.H != 99.0]
                    if target == "DPD":
                        owgframe = owgframe[owgframe.DPD != 99.0]
                    if target == "MWD":
                        owgframe = owgframe[owgframe.MWDIR != 999]
                else:
                    timecounter = timecounter + 1
            else:
                outofrangecounter = outofrangecounter + 1
        except:
            failcounter = failcounter + 1
    
    # write final dataframe to csv
    added = len(owgframe.index)
    owgframe.to_csv(csvfile, index = False)    
    
    # display performance information
    print ("you have {} images during buoy deployment".format(rangecounter))
    print ("{} images outside of buoy deployment".format(outofrangecounter))
    print ("{} images do not have data within 30 minutes".format(timecounter))
    print ("{} images had NAN data from buoy".format(initaladdcounter-int(added)))
    print ("{} images failed".format(failcounter))
    print ("{} images added to {}".format(added, csvfile))
    
   
    
    return  
                   


### Run function


In [22]:
fn = "C:/njc/src/GLERL_contract/buoy_data/mcy2021.txt"
target = "WVHT"
csvfile = "C:/njc/src/GLERL_contract/buoy_data/mcy2021_prototype.csv"
directory = "D:/ReCON_imgs/mcy_total/2021"
waveframetocsv(fn, target, csvfile, directory, "America/Chicago")

       WVHT    DPD  MWD                  datetime       epoch
0      0.41   2.68   24 2021-05-08 22:00:00+00:00  1620511200
1      0.39   2.78   63 2021-05-08 22:10:00+00:00  1620511800
2      0.39   2.45   65 2021-05-08 22:20:00+00:00  1620512400
3      0.36   2.62   84 2021-05-08 22:30:00+00:00  1620513000
4      0.34   2.73   86 2021-05-08 22:40:00+00:00  1620513600
...     ...    ...  ...                       ...         ...
20897  0.28   2.11  224 2021-10-14 11:30:00+00:00  1634211000
20898  0.27   2.33  217 2021-10-14 11:40:00+00:00  1634211600
20899  0.25   2.32  215 2021-10-14 11:50:00+00:00  1634212200
20900  0.23   2.34  227 2021-10-14 12:10:00+00:00  1634213400
20901  0.23  99.00  216 2021-10-14 12:20:00+00:00  1634214000

[20902 rows x 5 columns]
Overwriting csv file
couldn't find file, making new one
you have 1130 images during buoy deployment
412 images outside of buoy deployment
45 images do not have data within 30 minutes
43 images had NAN data from buoy
0 images faile

This shows not only a huge increase in available data per trial, but also ensures much more accurate and higher quality data which can only better serve model performance. 

### Code for creating model assessment dataset

In [25]:
def sample_ran(csv, sample_csv, sample_percent):
    ''' This function selects for a percentage of random rows in the csv training dataset 
    and creates a new csv for model assessment while dropping them from the original csv
    
    csv : Path to the csv that the function initially reads
    validation_csv : Path and name of the csv to be created from csv
    sample_percent : decimal value of the percentage of rows to be sampled '''
    
    if os.path.exists(sample_csv) == True:
        return print ("File already exists")
    else:    
        # open csv as a pandas dataframe 
        df = pd.read_csv(csv, sep = ",")
        # randomly sample data
        df_sample = df.sample(n = int(sample_percent * len(df.index)))
        # drop data from original dataframe that has been sampled
        df = df[~df.id.isin(df_sample.id)]
   
        # export dataframes as csv files
        df.to_csv(csv, sep = ",", index = False) 
        df_sample.to_csv(validation_csv, sep = ",", index = False)
        print ("writing {}".format(sample_csv))
    
        return print ("{}% of {} ({} rows) sampled".format(sample_percent*100, csv, len(df_sample)))

In [27]:
csv = "C:/njc/src/GLERL_contract/buoy_data/mcy2021_prototype.csv"
sample_csv = "C:/njc/src/GLERL_contract/buoy_data/mcy2021_prototype(sample).csv"
sample_percent = .3
sample_ran(csv, sample_csv, sample_percent)

File already exists
